In [1]:
print("hello")

hello


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import re

In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from tabulate import tabulate
import time

# Setup Selenium with headless Chrome
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode.
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://groww.in/fixed-deposit/sbi-fd-interest-rates"
driver.get(url)

# Wait a bit for the page to finish loading its JavaScript content
time.sleep(5)

# Get the rendered HTML source
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Find the first two tables with the specified class
tables = soup.find_all("table", class_="tb10Table borderPrimary")[:2]

if not tables:
    print("No tables with the specified class were found.")
else:
    for idx, table in enumerate(tables, start=1):
        print(f"\n\nProcessing table #{idx}:")

        # Look for the nearest preceding div for context
        preceding_div = table.find_previous_sibling("div")
        if not preceding_div:
            preceding_div = table.find_previous("div")
        if preceding_div:
            print("Preceding div content:")
            print(preceding_div.get_text(" ", strip=True))
        else:
            print("No preceding div found.")

        # Extract headers from <thead>
        headers = []
        header_section = table.find("thead")
        if header_section:
            headers = [th.get_text(" ", strip=True) for th in header_section.find_all("th")]

        # Extract rows from <tbody>
        rows = []
        tbody = table.find("tbody")
        if tbody:
            for tr in tbody.find_all("tr"):
                row = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
                if row:
                    rows.append(row)
        else:
            for tr in table.find_all("tr"):
                row = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
                if row:
                    rows.append(row)
        
        print("\nTable data:")
        print(tabulate(rows, headers=headers, tablefmt="grid"))

# Always quit the driver at the end
driver.quit()




Processing table #1:
Preceding div content:
SBI FD Rates 2025 (Below 3 Cr) Tenure Regular customers Senior citizens 7 days – 45 days 3.50% 4.00% 46 days – 179 days 5.50% 6.00% 180 days – 210 days 6.25% 6.75% 211 days – 364 days 6.50% 7.00% 1 year – 1 year 364 days 6.80% 7.30% 2 years – 2 years 364 days 7.00% 7.50% 3 years – 4 years 364 days 6.75% 7.25% 5 years – 10 years 6.50% 7.50% 400 Days (Amrit Kalash Scheme) 7.10% 7.60%

Table data:
+--------------------------------+---------------------+-------------------+
| Tenure                         | Regular customers   | Senior citizens   |
+================================+=====================+===================+
| 7 days – 45 days               | 3.50%               | 4.00%             |
+--------------------------------+---------------------+-------------------+
| 46 days – 179 days             | 5.50%               | 6.00%             |
+--------------------------------+---------------------+-------------------+
| 180 days – 210 

In [1]:
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from tabulate import tabulate

# --------------------------
# 1. Set up Selenium with headless Chrome
# --------------------------
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://groww.in/fixed-deposit/sbi-fd-interest-rates"
driver.get(url)
time.sleep(5)  # Wait for the JavaScript to load content

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Find the two tables (first for "below 3 cr", second for "above 3 cr")
tables = soup.find_all("table", class_="tb10Table borderPrimary")[:2]

# Initialize dictionary that will hold our FD rates data organized by key.
fd_data = {
    "below 3 cr": [],
    "above 3 cr": []
}

# --------------------------
# 2. Define a helper function to extract headers and rows
#    and replace Unicode en-dash with the ASCII hyphen.
# --------------------------
def extract_rows(table):
    # Extract headers and replace \u2013 with '-'
    header_section = table.find("thead")
    headers = [
        th.get_text(" ", strip=True).replace("\u2013", "-")
        for th in header_section.find_all("th")
    ] if header_section else []
    
    # Extract rows and perform the same replacement
    rows = []
    tbody = table.find("tbody")
    if tbody:
        for tr in tbody.find_all("tr"):
            row = [
                td.get_text(" ", strip=True).replace("\u2013", "-")
                for td in tr.find_all("td")
            ]
            if row:
                rows.append(row)
    else:
        for tr in table.find_all("tr"):
            row = [
                td.get_text(" ", strip=True).replace("\u2013", "-")
                for td in tr.find_all("td")
            ]
            if row:
                rows.append(row)
    return headers, rows

# --------------------------
# 3. Process the first table (below 3 cr)
# --------------------------
if len(tables) > 0:
    headers, rows = extract_rows(tables[0])
    for r in rows:
        # Using zip avoids index errors if the number of columns varies.
        obj = dict(zip(headers, r))
        fd_data["below 3 cr"].append(obj)

# --------------------------
# 4. Process the second table (above 3 cr)
# --------------------------
if len(tables) > 1:
    headers, rows = extract_rows(tables[1])
    for r in rows:
        obj = dict(zip(headers, r))
        fd_data["above 3 cr"].append(obj)

# --------------------------
# 5. Display the tables using tabulate (optional)
# --------------------------
print("Below 3 cr Rates:")
print(tabulate(fd_data["below 3 cr"], headers="keys", tablefmt="grid"))
print("\nAbove 3 cr Rates:")
print(tabulate(fd_data["above 3 cr"], headers="keys", tablefmt="grid"))

# Always close the driver when finished
driver.quit()

# --------------------------
# 6. Save the data to a JSON file
# --------------------------
with open("sbi_fd_rates.json", "w") as f:
    json.dump(fd_data, f, indent=4)
print("\nData successfully saved to sbi_fd_rates.json\n")


# --------------------------
# 7. Define a function to get the interest rate based on input parameters.
# --------------------------
def get_interest_rate(amount, age, tenure):
    """
    Returns the interest rate for a given fixed deposit.

    Parameters:
      amount (int): The deposit amount in rupees.
      age (int): The customer's age.
      tenure (str): A substring to match the 'Tenure' field (e.g., "7 days").
      
    Logic:
      - Uses the "below 3 cr" table if the amount is less than 30,000,000 rupees.
      - Otherwise, it uses the "above 3 cr" table.
      - If the age is less than 60, the "Regular customers" rate is used;
        otherwise, the "Senior citizens" rate is selected.
    
    Returns:
      The interest rate as a string, or None if no matching row is found.
    """
    with open("sbi_fd_rates.json", "r") as f:
        rates = json.load(f)
    
    # Select the correct table based on the deposit amount.
    key = "below 3 cr" if amount < 30000000 else "above 3 cr"
    table = rates.get(key, [])
    
    # Determine which column to use based on age.
    column = "Regular customers" if age < 60 else "Senior citizens"
    
    # Look for the row where the tenure substring matches.
    for row in table:
        if tenure.lower() in row.get("Tenure", "").lower():
            return row.get(column)
    
    return None

# --------------------------
# 8. Example usage of get_interest_rate:
# --------------------------
# Example: deposit of 20,000,000 rupees, age 35, and tenure with "7 days" (which corresponds to "7 days - 45 days").
example_rate = get_interest_rate(amount=20000000, age=35, tenure="7 days")
print(f"Interest rate for amount 20,000,000, age 35 and tenure '7 days': {example_rate}")

another_example_rate = get_interest_rate(amount=50000000, age=65, tenure="180 days")
print(f"Interest rate for amount 50,000,000, age 65 and tenure '180 days': {another_example_rate}")


ModuleNotFoundError: No module named 'webdriver_manager'